``` python
# if you running at azure nootbook to prepare env
#!pip install lxml --ignore-installed lxml
#!pip install tushare==1.2.54 
```

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from greenseer.configuration import do_global_configuration
from greenseer.dataset.china_dataset import fetch_default_targets,stock_info
from greenseer.dataset.china_dataset import fetch_multi_report
from logging.config import fileConfig
import numpy as np
from sklearn.pipeline import Pipeline
from greenseer.preprocessing.transformers import regular_expression_column_filter, \
    pick_annual_report_china, append_industry_transform, re_sum_column_transform, re_percent_column_transform
do_global_configuration("config.properties")
fileConfig('logging_config.ini')

In [18]:

y = fetch_default_targets()
y.head(5)

,st
code,
605001,0
688233,0
000612,0
300270,0
002369,0


- 600519：茅台

In [19]:
reports = fetch_multi_report(stock_ids=np.array(["600519","600520"]))

600519  2020-03-31    18162473.0
        2019-12-31    18304237.0
        2019-09-30    16255041.0
        2019-06-30    16512165.0
        2019-03-31    15988818.0
                         ...    
600520  2007-03-31       48676.0
        2006-12-31       51764.0
        2006-09-30           NaN
        2006-06-30           NaN
        2006-03-31           NaN
Name: 负债和所有者权益(或股东权益)总计(万元), Length: 114, dtype: float64

In [20]:
subjects = [r'\w*应付\w*|\w*应收\w*|存货|货币资金|\w*生物\w*|\w*资产总计\w*']


prepare_pipeline = Pipeline([
        ('pick_annual_report', pick_annual_report_china()),
        ('pick_subject', regular_expression_column_filter(patterns=subjects,rename={"资产总计(万元)":"base"})),
        ('percent payable', re_percent_column_transform(numerator=[r'\w*应付\w*'], denominator=[r'base'],new_name='payable')),
        ('percent receivable', re_percent_column_transform(numerator=[r'\w*应收\w*'], denominator=[r'base'],new_name='receivable')),
        ('append industry',append_industry_transform()),
])

data = prepare_pipeline.fit_transform(reports)

In [21]:
summary_by_category = data[['industry','receivable','payable',"base"]].groupby("industry")
summary_by_category.describe()

receivable                                                    \
              count      mean       std       min       25%       50%   
industry                                                                
机械基件           14.0  0.200669  0.049914  0.074252  0.177699  0.204306   
白酒             14.0  0.022640  0.024012  0.005991  0.010960  0.014486   

                             payable            ...                      base  \
               75%       max   count      mean  ...       75%       max count   
industry                                        ...                             
机械基件      0.216636  0.275299    14.0  0.209206  ...  0.278445  0.466148  14.0   
白酒        0.024627  0.100957    14.0  0.049961  ...  0.053526  0.073907  14.0   

                                                                       \
                  mean           std       min         25%        50%   
industry                                                                
机械基件      6.547693e+04  2.040822e+04   36318.0    48943.25    60326.5   
白酒        6.850894e+06  5.805926e+06  957079.0  2122411.00  5022618.0   

                                   
                  75%         max  
industry                           
机械基件         80756.25     96731.0  
白酒        10627627.00  18304237.0  

[2 rows x 24 columns]

In [22]:
data


专项应付款(万元)  公益性生物资产(万元)  其他应付款(万元)  其他应收款(万元)  内部应付款(万元)  \
code   release_at                                                            
600520 2019-12-31        0.0          0.0      494.0        0.0        0.0   
       2018-12-31        0.0          0.0      402.0        0.0        0.0   
       2017-12-31        0.0          0.0      456.0      392.0        0.0   
       2016-12-31        0.0          0.0      396.0      542.0        0.0   
       2015-12-31      158.0          0.0     1063.0      963.0        0.0   
       2014-12-31      157.0          0.0      515.0     2175.0        0.0   
       2013-12-31      115.0          0.0      952.0      878.0        0.0   
       2012-12-31       49.0          0.0      725.0      902.0        0.0   
       2011-12-31        0.0          0.0      528.0     1413.0        0.0   
       2010-12-31        0.0          0.0      993.0      824.0        0.0   
       2009-12-31        0.0          0.0     1102.0      682.0        0.0   
       2008-12-31        0.0          0.0      734.0     1042.0        0.0   
       2007-12-31        0.0          0.0      228.0     1239.0        0.0   
       2006-12-31        0.0          0.0       18.0     5778.0        0.0   
600519 2019-12-31        0.0          0.0   314263.0     7654.0        0.0   
       2018-12-31        0.0          0.0   336200.0     5000.0        0.0   
       2017-12-31     1557.0          0.0   303995.0     3132.0        0.0   
       2016-12-31     1557.0          0.0   172464.0     7723.0        0.0   
       2015-12-31     1557.0          0.0   142314.0     4822.0        0.0   
       2014-12-31     1777.0          0.0   123189.0     8089.0        0.0   
       2013-12-31     1777.0          0.0   160469.0    11957.0        0.0   
       2012-12-31     1777.0          0.0   138998.0    13797.0        0.0   
       2011-12-31     1677.0          0.0    91612.0     4729.0        0.0   
       2010-12-31     1000.0          0.0    81888.0     5910.0        0.0   
       2009-12-31     1000.0          0.0    71083.0     9600.0        0.0   
       2008-12-31        0.0          0.0    57591.0     8260.0        0.0   
       2007-12-31        0.0          0.0    42350.0     8611.0        0.0   
       2006-12-31        0.0          0.0    30003.0     5668.0        0.0   

                   内部应收款(万元)     存货(万元)  应付保证金(万元)  应付债券(万元)  应付分保账款(万元)  ...  \
code   release_at                                                         ...   
600520 2019-12-31        0.0    11271.0        0.0       0.0         0.0  ...   
       2018-12-31        0.0     9927.0        0.0       0.0         0.0  ...   
       2017-12-31        0.0    10489.0        0.0       0.0         0.0  ...   
       2016-12-31        0.0     9931.0        0.0       0.0         0.0  ...   
       2015-12-31        0.0     8865.0        0.0       0.0         0.0  ...   
       2014-12-31        0.0     8891.0        0.0       0.0         0.0  ...   
       2013-12-31        0.0     9566.0        0.0       0.0         0.0  ...   
       2012-12-31        0.0     7183.0        0.0       0.0         0.0  ...   
       2011-12-31        0.0     8462.0        0.0       0.0         0.0  ...   
       2010-12-31        0.0     7582.0        0.0       0.0         0.0  ...   
       2009-12-31        0.0     5045.0        0.0       0.0         0.0  ...   
       2008-12-31        0.0     8749.0        0.0       0.0         0.0  ...   
       2007-12-31        0.0     9151.0        0.0       0.0         0.0  ...   
       2006-12-31        0.0    10699.0        0.0       0.0         0.0  ...   
600519 2019-12-31        0.0  2528492.0        0.0       0.0         0.0  ...   
       2018-12-31        0.0  2350695.0        0.0       0.0         0.0  ...   
       2017-12-31        0.0  2205748.0        0.0       0.0         0.0  ...   
       2016-12-31        0.0  2062225.0        0.0       0.0         0.0  ...   
       2015-12-31        0.0  1801330.0        0.0       0